### Scimap framework - spatial interaction analysis
Quantify the spatial interactions between cell types by looking at co-localization outside of random chance.

In [ ]:
from functions import glasbey
import sklearn
from scipy.stats import ttest_ind
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import matplotlib as mpl
from anndata import AnnData
import anndata as ad
import scimap as sm
import pandas as pd
import numpy as np
import scanpy as sc
import igraph as ig
import seaborn as sns
import colorcet as cc
import copy
import os

# read in data
data_path = "gs://mayo-nguyen-transplant/nduarte-csv/adata.h5ad"
adata = ad.read_h5ad(data_path)

metadata = ['Image', 'Object ID', 'Classification', 'Parent', 'Centroid X µm', 'Centroid Y µm', 'Leiden', 'phenotype', 'Timepoint']
markers = ['DAPI', 'CD44', 'HLA-DR', 'CD4', 'IFNG', 'Ki67', 'CD107a', 'CD45', 'CD20', 'CD40', 'CD8', 'Pan-Cytokeratin', 'CD68', 'HLA-A', 'CD79a', 'CD45RO', 'CD21', 'CD11c', 'HLA-E', 'IDO1', 'CD14', 'CD56', 'VISTA', 'FOXP3', 'Granzyme B', 'PCNA', 'T-bet/TBX21', 'PD-L1', 'TOX', 'PD-1', 'CD38', 'ICOS', 'CD39', 'LAG3', 'TCF-1', 'CD3e']

In [ ]:
# make a pre DLN adata object and a post DLN one so can plot them separately
pre = adata.obs['Timepoint'] == 'DLN pre'
adata_pre = adata[pre].copy()
post = adata.obs['Timepoint'] == 'DLN post'
adata_post = adata[post].copy()
print(adata_pre)

In [ ]:
# pre interactions between cell types
adata_pre = sm.tl.spatial_interaction (adata_pre, 
                                   x_coordinate='Centroid X µm',
                                   y_coordinate='Centroid Y µm',
                                   method='radius', 
                                   imageid='Parent',
                                   radius=200, 
                                   permutation=100,
                                   label='spatial_interaction_radius')

In [ ]:
# pre interaction heatmap
sm.pl.spatial_interaction(adata_pre, 
                          spatial_interaction='spatial_interaction_radius',
                          p_val=0.01, cmap='plasma',
                        #   row_cluster=True, col_cluster=True,
                          linewidths=0.75, linecolor='black')

In [ ]:
# post interactions between cell types
adata_post = sm.tl.spatial_interaction (adata_post, 
                                   x_coordinate='Centroid X µm',
                                   y_coordinate='Centroid Y µm',
                                   method='radius', 
                                   imageid='Parent',
                                   radius=200, 
                                   permutation=100,
                                   label='spatial_interaction_radius')

In [ ]:
# post interaction heatmap
sm.pl.spatial_interaction(adata_post, 
                          spatial_interaction='spatial_interaction_radius',
                          p_val=0.01, cmap='viridis',
                        #   row_cluster=True, col_cluster=True,
                          linewidths=0.75, linecolor='black')

### Proximity volume and density
- Proximity volume = number of interactions to total cells in dataset
- Proximity density = number of interactions to number of cells in interacting types (interactions relative to pop size)

In [ ]:
print(adata_pre.obs['Phenotype'].unique())

In [ ]:
# spatial pscore
interactions_oi = [('Ant Pres Macrophage' , 'Plasma B'), ('Inhib Monocyte', 'Th1 Helper T'), ('Act Macrophage', 'Prolif Cyt T'), ('T reg', 'Plasma B')]
for interaction in interactions_oi:
    adata =  sm.tl.spatial_pscore (adata,
                                   imageid='Parent',
                                   x_coordinate='Centroid X µm',
                                   y_coordinate='Centroid Y µm',
                                   proximity= [interaction[0], interaction[1]],
                                   score_by = 'Timepoint',
                                   phenotype='phenotype',
                                   method='radius',
                                   label=f'spatial_pscore_{interaction[0]}-{interaction[1]}',
                                   radius=200)
    sm.pl.spatial_pscore (adata, color='black', label=f'spatial_pscore_{interaction[0]}-{interaction[1]}', figsize=(12,8))

In [ ]:
# voronoi plots to visualize the spatial pscore within the tissue - red is region of interaction
d = adata.obs
for interaction in interactions_oi:
    for parent in d['Parent'].unique():
        a = d[d['Parent'] == parent]
        if (interaction[0] in a['phenotype'].unique()) and (interaction[1] in a['phenotype'].unique()):
            print(parent)
            print(f'spatial_pscore_{interaction[0]}-{interaction[1]}')
            sm.pl.voronoi(adata, 
                            imageid='Parent',
                            x_coordinate='Centroid X µm',
                            y_coordinate='Centroid Y µm',
                            flip_y=False,
                            subset=parent,
                            color_by=f'spatial_pscore_{interaction[0]}-{interaction[1]}', 
                            voronoi_edge_color = 'black',
                            size_max=3000,
                            voronoi_line_width = 0.3, 
                            voronoi_alpha = 0.8, 
                            overlay_points=None, 
                            plot_legend=True, 
                            legend_size=6,
                            )

### Identify regions of aggregation
Identify spatial clusters of cells with similar phenotypes throughout tissue - purity is the minimum percentage of similarity required among cells 

In [ ]:
adata = sm.tl.spatial_aggregate(adata, 
                                x_coordinate='Centroid X µm', 
                                y_coordinate='Centroid Y µm', 
                                imageid='Parent', 
                                method='radius', 
                                radius=200, 
                                purity=80, 
                                label='spatial_aggregate_radius')

In [ ]:
adata.obs.head()

In [ ]:
d = adata.obs
all_parents = d['Parent'].unique()
for parent in all_parents:
    ncells = len(d[d['Parent'] == parent].index)
    print(parent)
    sm.pl.voronoi(adata, 
                    imageid='Parent',
                    x_coordinate='Centroid X µm',
                    y_coordinate='Centroid Y µm',
                    flip_y=False,
                    subset=parent,
                    color_by='spatial_aggregate_radius', 
                    voronoi_edge_color = 'black',
                    voronoi_line_width = 0.3, 
                    voronoi_alpha = 0.8, 
                    size_max=3000,
                    overlay_points=None, 
                    plot_legend=True, 
                    legend_size=6)

In [ ]:
sp = "gs://mayo-nguyen-transplant/nduarte-csv/adata2.h5ad"
sp2 = "gs://mayo-nguyen-transplant/nduarte-csv/adata_pre.h5ad"
sp3 = "gs://mayo-nguyen-transplant/nduarte-csv/adata_post.h5ad"

adata.write(sp)
adata_pre.write(sp2)
adata_post.write(sp3)